In [1]:
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import numpy as np
import tensorflow as tf
import pylab
import matplotlib.pyplot as plt
import time
import sys

/Users/stebliankin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import path to working directory
workdir="/Users/stebliankin/Desktop/Data Science-CAP5768/project/CatDog-CNN-Tensorflow-OnSpark"
sys.path.append(workdir)

In [3]:
# Import local functions:
import utils_MNIST
import image_op
import conv_net

In [5]:
# Download MNIST data using utils_MNIST
# MNIST dataset: http://yann.lecun.com/exdb/mnist/
# Contains 60,000 training examples of hand written digits

# Downloading MNIST data using API from stanford-tensorflow-tutorial:
# https://github.com/chiphuyen/stanford-tensorflow-tutorials/blob/master/examples/utils.py
os.mkdir(workdir+"/data")
utils_MNIST.download_mnist(workdir+"/data"+"/mnist")



Successfully downloaded /Users/stebliankin/Desktop/Data Science-CAP5768/project/CatDog-CNN-Tensorflow-OnSpark/data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded /Users/stebliankin/Desktop/Data Science-CAP5768/project/CatDog-CNN-Tensorflow-OnSpark/data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded /Users/stebliankin/Desktop/Data Science-CAP5768/project/CatDog-CNN-Tensorflow-OnSpark/data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded /Users/stebliankin/Desktop/Data Science-CAP5768/project/CatDog-CNN-Tensorflow-OnSpark/data/mnist/t10k-labels-idx1-ubyte.gz


In [6]:
# Make subclass of Mnist Convolutional Neural Network
# Architecture of the network is specified in conv_net.py

class MnistConvNet(conv_net.ConvNet):
    def __init__(self):
        super().__init__()
        self.n_classes = 10
        self.skip_step = 1  # printing rate

    def get_data(self):
        with tf.name_scope("mnist_data"):
            train_data, test_data = utils_MNIST.get_mnist_dataset(self.batch_size)
            iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
            img, self.label = iterator.get_next()
            self.img = tf.reshape(img, shape=[-1, 28, 28, 1])
            self.train_init = iterator.make_initializer(train_data)
            self.test_init = iterator.make_initializer(test_data)

In [7]:

start = time.time()
print('start program')
model = MnistConvNet()
print('building a model')
model.build()
print('training')
model.train(n_epochs=1)
print("Done. Running time is {} min.".format((time.time() - start)/60))

start program
building a model
data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
training
Running session
Variables initialized
start training
Loss at step 0: 2.3461496829986572
Loss at step 1: 2.2322397232055664
Loss at step 2: 2.1596157550811768
Loss at step 3: 2.107093572616577
Loss at step 4: 2.0383567810058594
Loss at step 5: 1.9527785778045654
Loss at step 6: 1.8467276096343994
Loss at step 7: 1.7157974243164062
Loss at step 8: 1.727343201637268
Loss at step 9: 1.5459778308868408
Loss at step 10: 1.5593876838684082
Loss at step 11: 1.2825446128845215
Lo

Loss at step 200: 0.21663184463977814
Loss at step 201: 0.17924150824546814
Loss at step 202: 0.1040247231721878
Loss at step 203: 0.1167449951171875
Loss at step 204: 0.17923544347286224
Loss at step 205: 0.15166720747947693
Loss at step 206: 0.10298560559749603
Loss at step 207: 0.18532690405845642
Loss at step 208: 0.16458940505981445
Loss at step 209: 0.19695180654525757
Loss at step 210: 0.10009614378213882
Loss at step 211: 0.14505624771118164
Loss at step 212: 0.15224692225456238
Loss at step 213: 0.11710817366838455
Loss at step 214: 0.1473563313484192
Loss at step 215: 0.12357605993747711
Loss at step 216: 0.13983865082263947
Loss at step 217: 0.14291483163833618
Loss at step 218: 0.20268596708774567
Loss at step 219: 0.10364866256713867
Loss at step 220: 0.23038634657859802
Loss at step 221: 0.0901050865650177
Loss at step 222: 0.1331804245710373
Loss at step 223: 0.18586251139640808
Loss at step 224: 0.09338787198066711
Loss at step 225: 0.2009235918521881
Loss at step 226: 

Loss at step 417: 0.09720823168754578
Loss at step 418: 0.09612728655338287
Loss at step 419: 0.16111493110656738
Loss at step 420: 0.13721629977226257
Loss at step 421: 0.1441315859556198
Loss at step 422: 0.12562912702560425
Loss at step 423: 0.10143409669399261
Loss at step 424: 0.06752309948205948
Loss at step 425: 0.10629211366176605
Loss at step 426: 0.09668680280447006
Loss at step 427: 0.07562172412872314
Loss at step 428: 0.07993283867835999
Loss at step 429: 0.16552485525608063
Average loss at epoch 0: 0.24219494393571864
Took: 73.36825434764226 minutes
Accuracy at epoch 0: 0.9761669303797469 
Took: 5.612814903259277 seconds
Done. Running time is 73.58290253480276 min.


In [8]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))